In [1]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.8)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_test=normalize(X_test)
    return X_train,X_test,Y_train,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers#aa##
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_test,Y_train,Y_test=create_dataset('chest_xray_lower_dim2')
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)

In [23]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu',kernel_regularizer=regularizers.L2(0.01))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [24]:
train(60,200,3,3)

Epoch 1/60
57/57 - 92s - loss: 1.6702 - accuracy: 0.4745 - val_loss: 1.3908 - val_accuracy: 0.4952 - 92s/epoch - 2s/step
Epoch 2/60
57/57 - 86s - loss: 1.3199 - accuracy: 0.4745 - val_loss: 1.2350 - val_accuracy: 0.4952 - 86s/epoch - 2s/step
Epoch 3/60
57/57 - 86s - loss: 1.2011 - accuracy: 0.4745 - val_loss: 1.1462 - val_accuracy: 0.4952 - 86s/epoch - 2s/step
Epoch 4/60
57/57 - 87s - loss: 1.1116 - accuracy: 0.4873 - val_loss: 1.0390 - val_accuracy: 0.5543 - 87s/epoch - 2s/step
Epoch 5/60
57/57 - 86s - loss: 1.0162 - accuracy: 0.5331 - val_loss: 0.9562 - val_accuracy: 0.5603 - 86s/epoch - 2s/step
Epoch 6/60
57/57 - 86s - loss: 0.9390 - accuracy: 0.5586 - val_loss: 0.8636 - val_accuracy: 0.6845 - 86s/epoch - 2s/step
Epoch 7/60
57/57 - 85s - loss: 0.7891 - accuracy: 0.6757 - val_loss: 0.7296 - val_accuracy: 0.6962 - 85s/epoch - 1s/step
Epoch 8/60
57/57 - 88s - loss: 0.7143 - accuracy: 0.7009 - val_loss: 0.6746 - val_accuracy: 0.7335 - 88s/epoch - 2s/step
Epoch 9/60
57/57 - 85s - loss: 0

In [38]:
#kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None)
def train(epochs_num,batch_len,conv_layers,neur_layers):
    X_trainR,XX,Y_trainR,YY=train_test_split(
    X_train,Y_train, test_size=0.05, random_state=20)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=2560
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None),kernel_regularizer=regularizers.L2(0.01))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=inputs*30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_trainR,Y_trainR,validation_split=0.2,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [39]:
train(60,200,3,3)

Epoch 1/60
54/54 - 106s - loss: 1.1004 - accuracy: 0.4707 - val_loss: 1.0750 - val_accuracy: 0.4734 - 106s/epoch - 2s/step
Epoch 2/60
54/54 - 81s - loss: 1.0625 - accuracy: 0.4750 - val_loss: 1.0581 - val_accuracy: 0.4734 - 81s/epoch - 2s/step
Epoch 3/60
54/54 - 82s - loss: 1.0565 - accuracy: 0.4750 - val_loss: 1.0569 - val_accuracy: 0.4734 - 82s/epoch - 2s/step
Epoch 4/60
54/54 - 81s - loss: 1.0558 - accuracy: 0.4750 - val_loss: 1.0568 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 5/60
54/54 - 81s - loss: 1.0557 - accuracy: 0.4750 - val_loss: 1.0569 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 6/60
54/54 - 82s - loss: 1.0558 - accuracy: 0.4750 - val_loss: 1.0568 - val_accuracy: 0.4734 - 82s/epoch - 2s/step
Epoch 7/60
54/54 - 81s - loss: 1.0557 - accuracy: 0.4750 - val_loss: 1.0569 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 8/60
54/54 - 81s - loss: 1.0558 - accuracy: 0.4750 - val_loss: 1.0570 - val_accuracy: 0.4734 - 81s/epoch - 1s/step
Epoch 9/60
54/54 - 81s - loss:

In [5]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.33,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [6]:
train(60,150,3,3)

Epoch 1/60
63/63 - 71s - loss: 1.0663 - accuracy: 0.4666 - val_loss: 1.0583 - val_accuracy: 0.4696 - 71s/epoch - 1s/step
Epoch 2/60
63/63 - 72s - loss: 1.0267 - accuracy: 0.4924 - val_loss: 0.9574 - val_accuracy: 0.5330 - 72s/epoch - 1s/step
Epoch 3/60
63/63 - 72s - loss: 0.9480 - accuracy: 0.5405 - val_loss: 0.9385 - val_accuracy: 0.5448 - 72s/epoch - 1s/step
Epoch 4/60
63/63 - 71s - loss: 0.9325 - accuracy: 0.5484 - val_loss: 0.9346 - val_accuracy: 0.5496 - 71s/epoch - 1s/step
Epoch 5/60
63/63 - 71s - loss: 0.9196 - accuracy: 0.5533 - val_loss: 0.9114 - val_accuracy: 0.5563 - 71s/epoch - 1s/step
Epoch 6/60
63/63 - 72s - loss: 0.9095 - accuracy: 0.5582 - val_loss: 0.9048 - val_accuracy: 0.5625 - 72s/epoch - 1s/step
Epoch 7/60
63/63 - 72s - loss: 0.9019 - accuracy: 0.5604 - val_loss: 0.9151 - val_accuracy: 0.5614 - 72s/epoch - 1s/step
Epoch 8/60
63/63 - 71s - loss: 0.9050 - accuracy: 0.5621 - val_loss: 0.8936 - val_accuracy: 0.5649 - 71s/epoch - 1s/step
Epoch 9/60
63/63 - 71s - loss: 0

In [7]:
train(60,300,3,3)

Epoch 1/60


KeyboardInterrupt: 

#### Let's try with 200x200


In [1]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.8)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_test=normalize(X_test)
    return X_train,X_test,Y_train,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_test,Y_train,Y_test=create_dataset('chest_xray_lower_dim200x200')
X_train=X_train.reshape(-1,200,200,1)
X_test=X_test.reshape(-1,200,200,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)

"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=200*200
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.33,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(60,150,4,3)
print("---------")
train(60,300,4,3)
print("---------")
train(60,500,4,3)

Epoch 1/60
63/63 - 379s - loss: 1.0642 - accuracy: 0.4740 - val_loss: 1.0557 - val_accuracy: 0.4787 - 379s/epoch - 6s/step
Epoch 2/60
63/63 - 366s - loss: 1.0432 - accuracy: 0.4914 - val_loss: 0.9698 - val_accuracy: 0.5392 - 366s/epoch - 6s/step
Epoch 3/60
63/63 - 369s - loss: 0.9513 - accuracy: 0.5412 - val_loss: 0.9269 - val_accuracy: 0.5464 - 369s/epoch - 6s/step
Epoch 4/60
63/63 - 394s - loss: 0.7875 - accuracy: 0.6397 - val_loss: 0.7058 - val_accuracy: 0.6779 - 394s/epoch - 6s/step
Epoch 5/60
63/63 - 374s - loss: 0.6438 - accuracy: 0.7117 - val_loss: 0.6233 - val_accuracy: 0.7066 - 374s/epoch - 6s/step
Epoch 6/60
63/63 - 370s - loss: 0.6050 - accuracy: 0.7266 - val_loss: 0.6027 - val_accuracy: 0.7245 - 370s/epoch - 6s/step
Epoch 7/60
63/63 - 357s - loss: 0.5864 - accuracy: 0.7444 - val_loss: 0.5890 - val_accuracy: 0.7413 - 357s/epoch - 6s/step
Epoch 8/60
63/63 - 368s - loss: 0.5598 - accuracy: 0.7624 - val_loss: 0.5680 - val_accuracy: 0.7605 - 368s/epoch - 6s/step
Epoch 9/60
63/63

KeyboardInterrupt: 

In [ ]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=200*200
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.5,epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(60,200,4,3)

#The score was 79.1%

#### We would try with bigger batches but the memory can't handle it with this power we return 100x100

In [4]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.8)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_test=normalize(X_test)
    return X_train,X_test,Y_train,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_test,Y_train,Y_test=create_dataset('chest_xray_lower_dim2')
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)

"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.33,epochs=epochs_num,batch_size=batch_len,verbose=1,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(60,300,3,3)
print("---------")
train(60,500,3,3)

Epoch 1/60
32/32 [==============================] - 78s 2s/step - loss: 1.0671 - accuracy: 0.4681 - val_loss: 1.0637 - val_accuracy: 0.4560
Epoch 2/60
32/32 [==============================] - 72s 2s/step - loss: 1.0227 - accuracy: 0.5062 - val_loss: 0.9804 - val_accuracy: 0.5248
Epoch 3/60
32/32 [==============================] - 74s 2s/step - loss: 0.9510 - accuracy: 0.5431 - val_loss: 0.9376 - val_accuracy: 0.5382
Epoch 4/60
32/32 [==============================] - 73s 2s/step - loss: 0.9293 - accuracy: 0.5499 - val_loss: 0.9270 - val_accuracy: 0.5423
Epoch 5/60
32/32 [==============================] - 72s 2s/step - loss: 0.9140 - accuracy: 0.5559 - val_loss: 0.9210 - val_accuracy: 0.5492
Epoch 6/60
32/32 [==============================] - 72s 2s/step - loss: 0.9065 - accuracy: 0.5604 - val_loss: 0.9165 - val_accuracy: 0.5474
Epoch 7/60
32/32 [==============================] - 73s 2s/step - loss: 0.9033 - accuracy: 0.5631 - val_loss: 0.9044 - val_accuracy: 0.5543
Epoch 8/60
32/32 [==

#### Now let's try to increase the number of epochs to see if it is better

In [5]:
train(200,150,3,3)

Epoch 1/200
63/63 [==============================] - 81s 1s/step - loss: 1.0603 - accuracy: 0.4751 - val_loss: 1.0683 - val_accuracy: 0.4560
Epoch 2/200
63/63 [==============================] - 78s 1s/step - loss: 1.0459 - accuracy: 0.4771 - val_loss: 1.0492 - val_accuracy: 0.4582
Epoch 3/200
63/63 [==============================] - 75s 1s/step - loss: 0.9608 - accuracy: 0.5345 - val_loss: 0.9075 - val_accuracy: 0.5354
Epoch 4/200
63/63 [==============================] - 81s 1s/step - loss: 0.7762 - accuracy: 0.6587 - val_loss: 0.6989 - val_accuracy: 0.6695
Epoch 5/200
63/63 [==============================] - 75s 1s/step - loss: 0.6599 - accuracy: 0.7053 - val_loss: 0.6632 - val_accuracy: 0.7031
Epoch 6/200
63/63 [==============================] - 75s 1s/step - loss: 0.6270 - accuracy: 0.7259 - val_loss: 0.6370 - val_accuracy: 0.7214
Epoch 7/200
63/63 [==============================] - 93s 1s/step - loss: 0.6047 - accuracy: 0.7404 - val_loss: 0.6187 - val_accuracy: 0.7296
Epoch 8/200
6

In [6]:
train(100,300,3,3)

Epoch 1/100
32/32 [==============================] - 88s 3s/step - loss: 1.0678 - accuracy: 0.4708 - val_loss: 1.0673 - val_accuracy: 0.4560
Epoch 2/100
32/32 [==============================] - 76s 2s/step - loss: 1.0404 - accuracy: 0.4891 - val_loss: 1.0048 - val_accuracy: 0.5151
Epoch 3/100
32/32 [==============================] - 75s 2s/step - loss: 0.9638 - accuracy: 0.5364 - val_loss: 0.9418 - val_accuracy: 0.5295
Epoch 4/100
32/32 [==============================] - 76s 2s/step - loss: 0.9286 - accuracy: 0.5492 - val_loss: 0.9334 - val_accuracy: 0.5332
Epoch 5/100
32/32 [==============================] - 76s 2s/step - loss: 0.9121 - accuracy: 0.5541 - val_loss: 0.9266 - val_accuracy: 0.5388
Epoch 6/100
32/32 [==============================] - 75s 2s/step - loss: 0.9046 - accuracy: 0.5575 - val_loss: 0.9031 - val_accuracy: 0.5474
Epoch 7/100
32/32 [==============================] - 79s 2s/step - loss: 0.8762 - accuracy: 0.5665 - val_loss: 0.8592 - val_accuracy: 0.5578
Epoch 8/100
3

In [14]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        wc=[100*100*98,49*49*40,23*23*40,10*10*40]
        for i in range(conv_layers):
            wc[i]=math.sqrt(wc[i])
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/wc[i],maxval=1/wc[i], seed=None))(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        wc[3]=math.sqrt(wc[3])
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/wc[3],maxval=1/wc[3], seed=None))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=2560
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/w,maxval=1/w, seed=None))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.33,epochs=epochs_num,batch_size=batch_len,verbose=1,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [15]:
train(100,300,3,3)

Epoch 1/100
32/32 [==============================] - 78s 2s/step - loss: 1.0737 - accuracy: 0.4678 - val_loss: 1.0681 - val_accuracy: 0.4560
Epoch 2/100
32/32 [==============================] - 77s 2s/step - loss: 1.0542 - accuracy: 0.4771 - val_loss: 1.0676 - val_accuracy: 0.4560
Epoch 3/100
32/32 [==============================] - 76s 2s/step - loss: 1.0553 - accuracy: 0.4771 - val_loss: 1.0667 - val_accuracy: 0.4560
Epoch 4/100
32/32 [==============================] - 78s 2s/step - loss: 1.0541 - accuracy: 0.4771 - val_loss: 1.0667 - val_accuracy: 0.4560
Epoch 5/100
32/32 [==============================] - 76s 2s/step - loss: 1.0542 - accuracy: 0.4771 - val_loss: 1.0671 - val_accuracy: 0.4560
Epoch 6/100
32/32 [==============================] - 77s 2s/step - loss: 1.0547 - accuracy: 0.4771 - val_loss: 1.0666 - val_accuracy: 0.4560
Epoch 7/100
32/32 [==============================] - 76s 2s/step - loss: 1.0543 - accuracy: 0.4771 - val_loss: 1.0669 - val_accuracy: 0.4560
Epoch 8/100
3

In [19]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.8)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_test=normalize(X_test)
    return X_train,X_test,Y_train,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_test,Y_train,Y_test=create_dataset('chest_xray_lower_dim2')
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)

"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.5,epochs=epochs_num,batch_size=batch_len,verbose=1,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,200,3,3)

Epoch 1/100
36/36 [==============================] - 71s 2s/step - loss: 1.0720 - accuracy: 0.4596 - val_loss: 1.0501 - val_accuracy: 0.4889
Epoch 2/100
36/36 [==============================] - 66s 2s/step - loss: 1.0317 - accuracy: 0.4906 - val_loss: 0.9439 - val_accuracy: 0.5512
Epoch 3/100
36/36 [==============================] - 68s 2s/step - loss: 0.9534 - accuracy: 0.5316 - val_loss: 0.9184 - val_accuracy: 0.5633
Epoch 4/100
36/36 [==============================] - 67s 2s/step - loss: 0.9261 - accuracy: 0.5414 - val_loss: 0.9269 - val_accuracy: 0.5542
Epoch 5/100
36/36 [==============================] - 67s 2s/step - loss: 0.9242 - accuracy: 0.5404 - val_loss: 0.9031 - val_accuracy: 0.5724
Epoch 6/100
36/36 [==============================] - 66s 2s/step - loss: 0.9122 - accuracy: 0.5485 - val_loss: 0.9068 - val_accuracy: 0.5643
Epoch 7/100
36/36 [==============================] - 66s 2s/step - loss: 0.9118 - accuracy: 0.5484 - val_loss: 0.9039 - val_accuracy: 0.5686
Epoch 8/100
3

NameError: name 'CategoricalTruePositives' is not defined

#### Now let's set monitor=loss to see overfitting with augmented data

In [1]:
#next time patience=2 node='loss' validation_split=0.33
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.8)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_test=normalize(X_test)
    return X_train,X_test,Y_train,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_test,Y_train,Y_test=create_dataset('chest_xray_lower_dim2')
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)

"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.33,epochs=epochs_num,batch_size=batch_len,verbose=1,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,300,3,3)

Epoch 1/100
32/32 [==============================] - 84s 2s/step - loss: 1.0697 - accuracy: 0.4638 - val_loss: 1.0473 - val_accuracy: 0.4864
Epoch 2/100
32/32 [==============================] - 72s 2s/step - loss: 1.0593 - accuracy: 0.4726 - val_loss: 1.0385 - val_accuracy: 0.4864
Epoch 3/100
32/32 [==============================] - 72s 2s/step - loss: 1.0004 - accuracy: 0.5062 - val_loss: 0.9291 - val_accuracy: 0.5522
Epoch 4/100
32/32 [==============================] - 72s 2s/step - loss: 0.9469 - accuracy: 0.5397 - val_loss: 0.9156 - val_accuracy: 0.5589
Epoch 5/100
32/32 [==============================] - 71s 2s/step - loss: 0.9333 - accuracy: 0.5456 - val_loss: 0.9143 - val_accuracy: 0.5643
Epoch 6/100
32/32 [==============================] - 72s 2s/step - loss: 0.9196 - accuracy: 0.5529 - val_loss: 0.8930 - val_accuracy: 0.5768
Epoch 7/100
32/32 [==============================] - 71s 2s/step - loss: 0.9053 - accuracy: 0.5614 - val_loss: 0.8820 - val_accuracy: 0.5821
Epoch 8/100
3

In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    #callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.33,epochs=epochs_num,batch_size=batch_len,verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,300,3,3)

Epoch 1/100
32/32 [==============================] - 97s 3s/step - loss: 1.0697 - accuracy: 0.4649 - val_loss: 1.0484 - val_accuracy: 0.4864
Epoch 2/100
32/32 [==============================] - 81s 3s/step - loss: 1.0511 - accuracy: 0.4726 - val_loss: 1.0175 - val_accuracy: 0.5004
Epoch 3/100
32/32 [==============================] - 94s 3s/step - loss: 0.9848 - accuracy: 0.5248 - val_loss: 0.9366 - val_accuracy: 0.5520
Epoch 4/100
32/32 [==============================] - 76s 2s/step - loss: 0.9514 - accuracy: 0.5346 - val_loss: 0.9151 - val_accuracy: 0.5584
Epoch 5/100
32/32 [==============================] - 75s 2s/step - loss: 0.9284 - accuracy: 0.5442 - val_loss: 0.9095 - val_accuracy: 0.5694
Epoch 6/100
32/32 [==============================] - 74s 2s/step - loss: 0.9169 - accuracy: 0.5519 - val_loss: 0.8900 - val_accuracy: 0.5757
Epoch 7/100
32/32 [==============================] - 76s 2s/step - loss: 0.9023 - accuracy: 0.5537 - val_loss: 0.8754 - val_accuracy: 0.5746
Epoch 8/100
3

Epoch 59/100
32/32 [==============================] - 72s 2s/step - loss: 0.2834 - accuracy: 0.8851 - val_loss: 0.5862 - val_accuracy: 0.7820
Epoch 60/100
32/32 [==============================] - 72s 2s/step - loss: 0.2832 - accuracy: 0.8850 - val_loss: 0.6380 - val_accuracy: 0.7775
Epoch 61/100
32/32 [==============================] - 72s 2s/step - loss: 0.2729 - accuracy: 0.8864 - val_loss: 0.6086 - val_accuracy: 0.7682
Epoch 62/100
32/32 [==============================] - 72s 2s/step - loss: 0.2574 - accuracy: 0.8974 - val_loss: 0.6582 - val_accuracy: 0.7842
Epoch 63/100
32/32 [==============================] - 75s 2s/step - loss: 0.2660 - accuracy: 0.8915 - val_loss: 0.6383 - val_accuracy: 0.7691
Epoch 64/100
32/32 [==============================] - 72s 2s/step - loss: 0.2542 - accuracy: 0.8962 - val_loss: 0.6347 - val_accuracy: 0.7732
Epoch 65/100
32/32 [==============================] - 73s 2s/step - loss: 0.2504 - accuracy: 0.8978 - val_loss: 0.6404 - val_accuracy: 0.7868
Epoch 

Non-trainable params: 0
_________________________________________________________________
Test loss: 1.8972465991973877
Test accuracy: 0.7636518478393555


In [3]:
# epoch=32 steps_per_epoch=32

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    #callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2,mode='min')
    model=mlp_model()
    model.fit(X_train,Y_train,validation_split=0.33,steps_per_epoch=32,epochs=epochs_num,batch_size=batch_len,verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(32,300,3,3)

Epoch 1/32
32/32 [==============================] - 79s 2s/step - loss: 1.0676 - accuracy: 0.4726 - val_loss: 1.0477 - val_accuracy: 0.4864
Epoch 2/32
32/32 [==============================] - 73s 2s/step - loss: 1.0441 - accuracy: 0.4817 - val_loss: 0.9863 - val_accuracy: 0.5509
Epoch 3/32
32/32 [==============================] - 73s 2s/step - loss: 0.9768 - accuracy: 0.5301 - val_loss: 0.9371 - val_accuracy: 0.5556
Epoch 4/32
32/32 [==============================] - 72s 2s/step - loss: 0.9411 - accuracy: 0.5425 - val_loss: 0.9367 - val_accuracy: 0.5554
Epoch 5/32
32/32 [==============================] - 73s 2s/step - loss: 0.9293 - accuracy: 0.5472 - val_loss: 0.9533 - val_accuracy: 0.5401
Epoch 6/32
32/32 [==============================] - 72s 2s/step - loss: 0.9228 - accuracy: 0.5477 - val_loss: 0.9181 - val_accuracy: 0.5543
Epoch 7/32
32/32 [==============================] - 76s 2s/step - loss: 0.9098 - accuracy: 0.5489 - val_loss: 0.8751 - val_accuracy: 0.5686
Epoch 8/32
32/32 [==

#### This time augmentation only on training dataset

In [1]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,Y,file,aug):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code
    if "virus" in file:
        Y.append("0")
    elif "bacteria" in file:
        Y.append("1")
    else:
        Y.append("2")
    if aug:
        augmentation(X,Y)

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset
    
def create_dataset(directory):
    #before work
    files=glob.glob(directory+'/*.jpeg')
    ra.shuffle(files)
    X_train=[]
    Y_train=[]
    X_valid=[]
    Y_valid=[]
    X_test=[]
    Y_test=[]
    for file in files[0:int(len(files)*0.64)]:
        process(X_train,Y_train,file,True)
    for file in files[int(len(files)*0.64):int(len(files)*0.8)]:
        process(X_valid,Y_valid,file,False)    
    for file in files[int(len(files)*0.8):]:
        process(X_test,Y_test,file,False)        
    from numpy import asarray
    X_train=normalize(X_train)
    X_valid=normalize(X_valid)
    X_test=normalize(X_test)
    return X_train,X_valid,X_test,Y_train,Y_valid,Y_test

def augmentation(dataset,labels):
    last=len(labels)-1
    scale_out = skimage.transform.rescale(dataset[last], scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in = skimage.transform.rescale(dataset[last], scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    dataset.append(scale_out)
    labels.append(labels[last])
    dataset.append(scale_in)
    labels.append(labels[last])
    

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math
X_train,X_valid,X_test,Y_train,Y_valid,Y_test=create_dataset('chest_xray_lower_dim200x200')
X_valid=X_valid.reshape(-1,200,200,1)
X_test=X_test.reshape(-1,200,200,1)
Y_train= keras.utils.to_categorical(Y_train,3)
Y_valid=keras.utils.to_categorical(Y_valid,3)
Y_test = keras.utils.to_categorical(Y_test,3)

In [5]:
"""
next time
penalty 0.01
penalty last 0.001
reverse
"""
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    #X_train,XX,Y_train,YY=train_test_split(
    #dataset, labels, test_size=0.2, random_state=42)

    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,mode='min',restore_best_weights=True)
    model=mlp_model()
    model.fit(X_train,Y_train,validation_data=(X_valid,Y_valid),epochs=epochs_num,batch_size=batch_len,verbose=2,callbacks=[callback])
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])

In [6]:
train(100,150,4,3)

Epoch 1/100
75/75 - 458s - loss: 1.0602 - accuracy: 0.4761 - val_loss: 1.0552 - val_accuracy: 0.4781 - 458s/epoch - 6s/step
Epoch 2/100
75/75 - 462s - loss: 1.0550 - accuracy: 0.4761 - val_loss: 1.0379 - val_accuracy: 0.4781 - 462s/epoch - 6s/step
Epoch 3/100
75/75 - 463s - loss: 0.9765 - accuracy: 0.5285 - val_loss: 0.6552 - val_accuracy: 0.6916 - 463s/epoch - 6s/step
Epoch 4/100
75/75 - 466s - loss: 0.9149 - accuracy: 0.5547 - val_loss: 0.5878 - val_accuracy: 0.7407 - 466s/epoch - 6s/step
Epoch 5/100
75/75 - 682s - loss: 0.8986 - accuracy: 0.5629 - val_loss: 0.5579 - val_accuracy: 0.7609 - 682s/epoch - 9s/step
Epoch 6/100
75/75 - 738s - loss: 0.8901 - accuracy: 0.5687 - val_loss: 0.5555 - val_accuracy: 0.7748 - 738s/epoch - 10s/step
Epoch 7/100
75/75 - 785s - loss: 0.8889 - accuracy: 0.5673 - val_loss: 0.5370 - val_accuracy: 0.7780 - 785s/epoch - 10s/step
Epoch 8/100
75/75 - 908s - loss: 0.8801 - accuracy: 0.5734 - val_loss: 0.5293 - val_accuracy: 0.7866 - 908s/epoch - 12s/step
Epoch